In [1]:
pip install opencv-python tensorflow keras numpy pandas matplotlib scikit-learn


  Using cached scikit_learn-1.5.2-cp311-cp311-win_amd64.whl (11.0 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt


In [2]:
def load_dataset(real_dir, fake_dir, img_size=(128, 128)):
    data = []
    labels = []

    for img_path in os.listdir(real_dir):
        img = cv2.imread(os.path.join(real_dir, img_path))
        img = cv2.resize(img, img_size)
        data.append(img)
        labels.append(0)  # Real images labeled as 0

    for img_path in os.listdir(fake_dir):
        img = cv2.imread(os.path.join(fake_dir, img_path))
        img = cv2.resize(img, img_size)
        data.append(img)
        labels.append(1)  # Fake images labeled as 1

    data = np.array(data) / 255.0  # Normalize pixel values
    labels = np.array(labels)

    return data, labels


In [3]:
real_dir = 'C:/Users/Aditha Ayomal/Downloads/Compressed/archive_3/videos/Real'
fake_dir = 'C:/Users/Aditha Ayomal/Downloads/Compressed/archive_3/videos/Fake'
img_size = (128, 128)

data, labels = load_dataset(real_dir, fake_dir, img_size)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [4]:
def build_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_data_generators(real_dir, fake_dir, img_size, batch_size=32):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        directory=os.path.dirname(real_dir),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        directory=os.path.dirname(real_dir),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )

    return train_generator, validation_generator

batch_size = 32
train_gen, val_gen = create_data_generators(real_dir, fake_dir, img_size, batch_size)

model = build_model((img_size[0], img_size[1], 3))

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


Found 13147 images belonging to 2 classes.
Found 3286 images belonging to 2 classes.
Epoch 1/10


In [ ]:
model = build_model((img_size[0], img_size[1], 3))

history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")
